In [1]:
from IPython.display import display

from sympy import *
import numpy as np

x, y, z = symbols('x y z')
init_printing(use_unicode=True)

In [2]:
# No algorithm provided, I wrote this from the steps in the text
def lagrange_interpolate(
    vals: (list[float], list[float]),
    n: int,
    target: float,
) -> float:
    """
    TODO! Prioritize points closest to target
    
    Parameters
    ----------
    vals : (list[float], list[float])
        x, y values
    n : int
        L_n Lagrange interpolation to find.
    target : float
        Target x to calculate.
    """
    sol = 0
    xs, ys = vals
    for i in range(n+1):
        inter = 1
        for k in range(n+1):
            if k == i:
                continue
            inter *=  (target - xs[k])/(xs[i] - xs[k])
        sol += ys[i] * inter
    return sol

# Homework

## Exercise set 3.1

### 6b

Use appropriate Lagrange interpolating polynomials of degrees one, two, and three to approximate each of the following:

$f(0)$ if $f(-0.5)=1.93750, f(-0.25) = 1.33203, f(0.25) = 0.800781, f(0.5) = 0.687500$

**solution**

$L_0, f(0) \approx 0.7265600000000001$

$L_1, f(0) \approx 0.9531236666666667$

$L_2, f(0) \approx 0.984374$


In [3]:
# 3.1, 6b
vals = (
    [-0.5, -0.25, 0.25, 0.5],
    [1.93750, 1.33203, 0.800781, 0.687500],
)

for i in range(3):
    print(
        'L_{}'.format(i),
        lagrange_interpolate(vals, i + 1, 0),
    )

L_0 0.7265600000000001
L_1 0.9531236666666667
L_2 0.984374


### 8b

The data for Exercise 6 were generated using the following functions.
Use the error forumla to find a bound for the error, and compare the bound to the actual error for the cases $n=1$ and $n=2$.

$f(x)=x^4-x^3+x^2-x+1$

**solution**

$$\begin{align}
f^1(x)= 4x^3 + 3x^2 + 2x - 1 \\
f^2(x)= 12x^2 + 6x + 2\\
f^3(x)= 24x + 6 \\
f^4(x) = 24 \\
\end{align}$$

error form: $$\frac{24+6}{3!}(x+0.5)(x-0.25)(x-0.5)$$

absolute error $n=1, 0.70313$

bound: $$\frac{24+6}{3!}(1+0.5)(1-0.25)(1-0.5)=2.8125$$

absolute error $n=2, 14.76569$

bound: $$\frac{24+6}{3!}(2+0.5)(2-0.25)(2-0.5)=32.8125$$

In [4]:
# 3.1, 8b
P = lagrange_interpolate(vals, 3, x)
f = lambda x: x**4 - x**3 + x**2 - x + 1

print(abs(f(1) - P.subs({x: 1})))

print(abs(f(2) - P.subs({x: 2})))

0.703130000000002
14.7656900000000


In [5]:
# ALGORITHM 3.1

def neville_iterated_interpolation(
    vals: (list[float], list[float]),
    target: float,
):
    """
    Parameters
    ----------
    vals : (list[float], list[float])
        x, y values
    target : float
        Target x to calculate.
    """
    xs, ys = vals
    assert len(xs) == len(ys)
    Q = np.zeros((len(xs), len(ys)))
    Q[:,0] = ys
    for i in range(1, len(xs)):
        # display(Q)
        for j in range(1, i+1):
            Q[i][j] = ((target - xs[i-j]) * Q[i][j-1] - (target - xs[i]) * Q[i-1][j-1]) / (xs[i] - xs[i-j])
    return Q
# test from given answers in book
# neville_iterated_interpolation(([8.1, 8.3, 8.6, 8.7], [16.94410, 17.56492, 18.50515, 18.82091]), 8.4)
# neville_iterated_interpolation(([2.0, 2.2, 2.3], [0.6931, 0.7885, 0.8329]), 2.1)

## Exercise set 3.2

### 2a

Use Neville's method to obtain the approximations for Lagrange interpolating polynomials of degrees one, two, and three to approximate each of the following:

$f(0.43)$ if $f(0)=1, f(0.25) = 1.64872, f(0.5) = 2.71828, f(0.75) = 4.48169$

**solution**

$P_1, f(0.43) \approx 2.11580$

$P_2, f(0.43) \approx 2.37638$

$P_3, f(0.43) \approx 2.36060$

In [6]:
neville_iterated_interpolation(
    (
        [0, 0.25, 0.5, 0.75],
        [1, 1.64872, 2.71828, 4.48169],
    ),
    0.43,
)

array([[1.        , 0.        , 0.        , 0.        ],
       [1.64872   , 2.1157984 , 0.        , 0.        ],
       [2.71828   , 2.4188032 , 2.37638253, 0.        ],
       [4.48169   , 2.2245252 , 2.34886312, 2.36060473]])

### 6

Neville's method is used to approximate $f(0.5)$, giving the following table.

|||||
|---|---|---|---|
|$x_0=0$|$P_0=0$|||
|$x_1=0.4$|$P_1=2.8$|$P_{0,1}=3.5$||
|$x_2=0.7$|$P_2$|$P_{1,2}$|$P_{0,1,2}=\frac{27}{7}$|

Determine $P_2=f(0.7)$.

**solution**

$P_2, f(0.7) \approx 6.4$

In [7]:
neville_iterated_interpolation(
    (
        [0, 0.4, 0.7],
        [0, 2.8, 6.4],
    ),
    0.5,
)

array([[0.        , 0.        , 0.        ],
       [2.8       , 3.5       , 0.        ],
       [6.4       , 4.        , 3.85714286]])

In [8]:
# ALGORITHM 3.2

def newton_div_diff_interpolation(
    vals: (list[float], list[float]),
):
    """
    Parameters
    ----------
    vals : (list[float], list[float])
        x, y values
    """
    xs, ys = vals
    assert len(xs) == len(ys)
    F = np.zeros((len(xs), len(ys)))
    F[:,0] = ys
    for i in range(1, len(xs)):
        for j in range(1, i+1):
            F[i][j] = (F[i][j-1] - F[i-1][j-1])/(xs[i] - xs[i-j])
    return F


def newton_forward_diff(F, h, s, n):
    D = F.diagonal()
    assert len(D) >= n
    res = 0
    for i in range(n+1):
        inner = 1
        for j in range(i):
             inner *= (s - j)
        res += D[i] * pow(h, i) * inner
    return res


def newton_backward_diff(F, h, s, n):
    D = F[-1]
    assert len(D) >= n
    res = 0
    for i in range(n+1):
        inner = 1
        for j in range(i):
             inner *= (s + j)
        res += D[i] * pow(h, i) * inner
    return res

# test from book
F = newton_div_diff_interpolation(
    (
        [1.0, 1.3, 1.6, 1.9, 2.2],
        [0.7651977, 0.6200860, 0.4554022, 0.2818186, 0.1103623],
    ),
)
# display(F)
# display(newton_forward_diff(F, 0.3, 1/3, 4))
# display(newton_backward_diff(F, 0.3, -2/3, 4))

## Exercise set 3.3

### 4a

Use the Newton forward-difference formula to construct interpolating polynomials of degree one, two, and three for the following data.
Approximate the specified value using each of the polynomials.

$f(0.43)$ if $f(0)=1, f(0.25) = 1.64872, f(0.5) = 2.71828, f(0.75) = 4.48169$

**solution**

$s=\frac{1}{h}(x-x_0)=4*(0.43-0.25)=0.72$

$P_n(0.43)=P_n(0.25 + 0.72*0.25), h=0.25, s=0.72$

$P_1(0.43)\approx 1.46708$

$P_2(0.43)\approx 1.42466$

$P_3(0.43)\approx 1.43640$

In [9]:
F = newton_div_diff_interpolation(
    (
        [0, 0.25, 0.5, 0.75],
        [1, 1.64872, 2.71828, 4.48169],
    ),
)
for n in range(1, 4):
    v = newton_forward_diff(F, 0.25, 0.72, n)
    display(v)

### 5a

Use the Newton backward-difference formula to construct interpolating polynomials of degree one, two, and three for the following data.
Approximate the specified value using each of the polynomials.

$f(-\frac{1}{3})$ if $f(-0.75) = -0.07181250, f(-0.5) = -0.02475000, f(-0.25) = 0.33493750, f(0) = 1.10100000$

**solution**

$s=\frac{1}{h}(x-x_n)=4*(-\frac{1}{3}-0)=-\frac{4}{3}$

$P_n(-\frac{1}{3})=P_n(0.25 + 0.72*0.25), h=0.25, s=-\frac{4}{3}$

$P_1, f(-\frac{1}{3})\approx 0.07958$

$P_2, f(-\frac{1}{3})\approx 0.16989$

$P_3, f(-\frac{1}{3})\approx 0.17451$


In [10]:
F = newton_div_diff_interpolation(
    (
        [-0.75, -0.5, -0.25, 0],
        [-0.07181250, -0.02475000, 0.33493750, 1.10100000],
    ),
)
for n in range(1, 4):
    v = newton_backward_diff(F, 0.25, -4/3, n)
    display(v)

## Exercise set 3.4

### 2b

Use Theorem 3.9 or Algorithm 3.3 to construct an approximating polynomial for the following data.

|$x$|$f(x)$|$f'(x)$|
|---|---|---|
|-0.25|1.33203|0.437500|
|0.25|0.800781|-0.625000|

**solution**

Using the matrix approach from Bill Kinney's video.

$P(x)=7.74998x^3-1.0625x^2-1.54687x+1.13281$

In [11]:
# Hermite interpolation
x0 = -0.25
x1 = 0.25
data = np.array([
    [x0**3, x0**2, x0, 1],
    [x1**3, x1**2, x1, 1],
    [3*(x0**2), 2*x0, 1, 0],
    [3*(x1**2), 2*x1, 1, 0],
])
dep = np.array([1.33203, 0.800781, 0.437500, -0.625000])
np.linalg.solve(data, dep)

array([ 7.749984  , -1.0625    , -1.546872  ,  1.13281175])

### 4b

The data in Exercise 2 were generated using the following functions.
Use the polynomials constructed in Exercise 2 for the given value of $x$ to approximate $f(x)$, and calculate the absolute error.

$f(x)=x^4-x^3+x^2-x+1$; approximate $f(0)$.

**solution**

$f(0) = 1$

$P(0) = 1.13281$

Absolute error = $|1.13281 - 1| = 0.13281$

### 10

A car traveling along a straight road is clocked at a number of points.
The data from the observations are given in the following table, where the time is in seconds, the distance is in feet, and the speed is in feet per second.

| | | | | | |
|---|---|---|---|---|---|
|Time|0|3|5|8|13|
|Distance|0|255|383|623|993|
|Speed|75|77|80|74|72|


### a

Use a Hermite polynomial to predict the position of the car and its speed when $t=10s$.

**solution**

$P(10) \approx 969$ feet

In [12]:
from functools import partial, reduce

xs = [0, 3, 5, 8, 13]
ys = [0, 255, 383, 623, 993]
ps = [75, 77, 80, 74, 72]

# book example
# xs = [1.3, 1.6, 1.9]
# ys = [0.6200860, 0.4554022, 0.2818186]
# ps = [-0.5220232, -0.5698959, -0.5811571]
# [-0.00277469,  0.02403179, -0.01455608, -0.23521617, -0.00822922, 1.00194406]
# [-0.00277469 * pow(1.5, 5), 0.02403179 * pow(1.5, 4), -0.01455608 * pow(1.5, 3), -0.23521617 * pow(1.5, 2), -0.00822922 * 1.5 + 1.00194406]
# -0.00277469 * pow(1.5, 5) + 0.02403179 * pow(1.5, 4) - 0.01455608 * pow(1.5, 3) - 0.23521617 * pow(1.5, 2) - 0.00822922 * 1.5 + 1.00194406

# xs = [-0.25, 0.25]
# ys = [1.33203, 0.800781]
# ps = [0.437500, -0.625000]

# TODO: this is unreadable
f = lambda v, n: list(map(lambda i: pow(v, i), range(2*n-1, 0, -1)))
fs = list(map(partial(f, n=len(xs)), xs))
for l in fs:
    l.append(1)

f_prime = lambda v, n: list(map(lambda i: i * pow(v, i-1), range(2*n-1, 1, -1)))
fps = list(map(partial(f_prime, n=len(xs)), xs))
for l in fps:
    l.extend([1, 0])

fs.extend(fps)
data = np.array(fs)
dep = np.array(ys + ps)

sol = np.linalg.solve(data, dep)
res = 0
display(sol)
res = lambda v: sum(map(lambda t: t[0] * pow(v, t[1]), zip(sol, range(len(sol)-1, -1, -1))))
for n in [0, 3, 5, 8, 10, 13]:
    print(n, res(n))

array([ 2.90887479e-04, -1.57371875e-02,  3.49679889e-01, -4.11806986e+00,
        2.76085933e+01, -1.04547591e+02,  2.04445136e+02, -1.55078092e+02,
        7.50000000e+01,  0.00000000e+00])

0 0.0
3 254.9999999999991
5 383.00000000001364
8 622.9999999998472
10 968.5028390989719
13 993.0000000011496


### b

Use the derivative of the Hermite polynomial to determine whether the car ever exceeds a 55 mi/h speed limit on the road.
If so, what is the first time the car exceeds this speed?

**solution**

$P'(x)=0.00261798731224428 x^{8} - 0.125897500379323 x^{7} + 2.44775922470074 x^{6} - 24.7084191795674 x^{5} + 138.042966448118 x^{4} - 418.190362399186 x^{3} + 613.335406517533 x^{2} - 310.156183925115 x + 75.0
0.00261798731224428 x^{8} - 0.125897500379323 x^{7} + 2.44775922470074 x^{6} - 24.7084191795674 x^{5} + 138.042966448118 x^{4} - 418.190362399186 x^{3} + 613.335406517533 x^{2} - 310.156183925115 x + 75.0$

The car first exceeds 55 mi/h at $t \approx 1.8$.

In [13]:
der = lambda v: sum(map(lambda t: t[1] * t[0] * pow(v, t[1]-1), zip(sol[:-1], range(len(sol)-1, 0, -1))))
print(latex(der(x)))
for n in [0, 3, 5, 8, 10, 13]:
    print(der(n))

0.00261798731224428 x^{8} - 0.125897500379323 x^{7} + 2.44775922470074 x^{6} - 24.7084191795674 x^{5} + 138.042966448118 x^{4} - 418.190362399186 x^{3} + 613.335406517533 x^{2} - 310.156183925115 x + 75.0
75.0
77.00000000000341
80.0000000000266
74.00000000006003
287.31506969756674
72.00000000325508


### c

What is the predicted maximum speed for the car?

**solution**

The maximum speed is approximately 201 mi/h at $t \approx 10.25s$.
This result (and the fact that the car is going backwards at some point in this interval) seems ludicrous.
I hope it is a consequence of using interpolation and not an error in my calculations, but I am not holding my breath on that.